In [64]:
def reconstruct_channel(compressed_channel):
    reconstructed_channel = compressed_channel["U"] @ np.diag(compressed_channel["S"]) @ compressed_channel["Vt"]
    reconstructed_channel = np.clip(reconstructed_channel, 0, 255).astype(np.uint8)
    return reconstructed_channel

In [65]:
def reconstruct_image(compressed_image):
    reconstructed_image=[]
    for idx, value in enumerate(compressed_image,start=0):
        reconstructed_image.append(reconstruct_channel(value))
    if len(reconstructed_image) > 1:
        return np.stack(reconstructed_image, axis=2)
    else:
        return reconstructed_image[0]

In [78]:
def compress_channel(channel , percent):
    U,S,Vt = np.linalg.svd(channel , full_matrices=True)
    explained_variance_ratio = (S**2) / np.sum(S**2)
    cumulative = np.cumsum(explained_variance_ratio)
    standardized_percent = np.clip((percent*(cumulative[-1]-cumulative[0]) + cumulative[0]), 0, 1)
    k = np.argmax(cumulative >= standardized_percent) + 1
    compressed_data = {
        "U": U[:, :k],
        "S": S[:k],
        "Vt": Vt[:k, :],
        "k": k,
        "cumulative":cumulative,
        "percent":percent
    }
    return compressed_data

In [67]:
def compress_image(image,variance_percent):
    compressed_image=[]
    if len(image.shape) == 2:
        compressed_image.append(compress_channel(image, variance_percent))
    else:  # RGB
        for i in range(0,image.shape[2]):
            compressed_channel=compress_channel(image[:, :, i], variance_percent)
            compressed_image.append(compressed_channel)
        
    return compressed_image

In [74]:
import numpy as np
import cv2
image = cv2.imread(r"Data/messi.jpg")
precentage = input("enter percent of pic preservence %") 
precentage=int(precentage)/100


enter percent of pic preservence % 90


### compression

In [75]:
stored_image = compress_image(image,precentage)
space_after_compression = 0
information_kept = 0
ch_n=0
for ch in stored_image:
        space_after_compression+=ch["U"].size+ch["S"].size+ch["Vt"].size
space_ratio = (space_after_compression / image.size)*100
print ("pic is compressed successfully with a percent of ", 100-space_ratio,"% ")

0.9543261042203866
0.9358463660203149
0.9771852602231705
pic is compressed successfully with a percent of  72.94185446206673 % 


### reconstruction

In [77]:
reconstructed_image=reconstruct_image(stored_image)
cv2.imshow("Reconstructed image",reconstructed_image)
cv2.imshow("Original image",image)

cv2.waitKey(0) 
cv2.destroyAllWindows()